In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import os
from datetime import date, time
from matplotlib import pyplot as plt
from dataclasses import replace
from ssrs import Simulator, Config

# Comparison of tracks at different locations in WY

Setting W winds and 8 m/s throghout. Using the pparoach of 8 squares on each side, migrating to the other direction.

Jul 5, 2022


In [ ]:
wspd = 8
wdir = 270

In [ ]:
# 25x25 km boxes selected by Dave; sent on Jun 30
box1 = ( -106.60, 41.90 )
box2 = ( -106.25, 42.23 )
box3 = ( -106.15, 41.88 )
box4 = ( -106.14, 42.91 )

In [ ]:
default_config = Config()

# Default WY parameters
default_config =replace(
    default_config, 
    fig_height=8.,
    sim_seed = -1,
    #southwest_lonlat = (-121.72, 37.53),
    region_width_km = (25.0, 25.0),
    sim_mode = 'uniform',
    uniform_windspeed = wspd,
    uniform_winddirn = wdir,
    uniform_windspeed_h = wspd,
    uniform_winddirn_h = wdir,
    href = 80.0,
    uniform_windspeed_href = wspd,
    uniform_winddirn_href = wdir,
    turbine_minimum_hubheight = 70.,
    track_direction = 10, # random value. It will be replaced
    track_count = 500,
    track_start_region = (2, 23, 0, 1), # xmin, xmax, ymin, ymax in km wrt box selected
    track_dirn_restrict = 2,
    track_stochastic_nu = 1.0,
    turbine_mrkr_size = 10.,
    updraft_threshold = 0.75,
    orographic_model = 'improved',
    slopeAspectMode = 'compute',
    resolution = 50.,
    resolution_terrain = 10.,
)

## Initialize SSRS objects for the 4 different boxes

In [ ]:
# Box1, improved model
wy_box1_improved_dict =replace(
    default_config, 
    run_name='wy_box1_improved',
    southwest_lonlat = box1,
    orographic_model = 'improved'
)

# Box1, original model
wy_box1_original_dict =replace(
    default_config, 
    run_name='wy_box1_original',
    southwest_lonlat = box1,
    orographic_model = 'original'
)

In [ ]:
# Box2, improved model
wy_box2_improved_dict =replace(
    default_config, 
    run_name='wy_box2_improved',
    southwest_lonlat = box2,
    orographic_model = 'improved'
)

# Box2, original model
wy_box2_original_dict =replace(
    default_config, 
    run_name='wy_box2_original',
    southwest_lonlat = box2,
    orographic_model = 'original'
)

In [ ]:
# Box3, improved model
wy_box3_improved_dict =replace(
    default_config, 
    run_name='wy_box3_improved',
    southwest_lonlat = box3,
    orographic_model = 'improved'
)

# Box3, original model
wy_box3_original_dict =replace(
    default_config, 
    run_name='wy_box3_original',
    southwest_lonlat = box3,
    orographic_model = 'original'
)

In [ ]:
# Box4, improved model
wy_box4_improved_dict =replace(
    default_config, 
    run_name='wy_box4_improved',
    southwest_lonlat = box4,
    orographic_model = 'improved'
)

# Box4, original model
wy_box4_original_dict =replace(
    default_config, 
    run_name='wy_box4_original',
    southwest_lonlat = box4,
    orographic_model = 'original'
)

## Simulate the tracks varying the entry point

In [ ]:
def runNdirections(sim_object_dict, N=8):
    
    # Get dimentions from object
    extentx = sim_object_dict.region_width_km[0]
    extenty = sim_object_dict.region_width_km[0]
    startboxsize = 1 # 1 km boxes

    # Get starting location information
    theta = np.deg2rad( np.linspace(0,360,N, endpoint=False) )  # 8 points = N, S, W, E, NW, NE, SW, SE

    # Get track direction wrt to each box.
    # First box is N (0), so track direction should be S (180), 3rd box is at E (90), so track direction should be W (270). Create this offset here
    dirns = np.rad2deg(theta)+180

    # create 1x1km starting boxes at the locations indicated by theta
    x = (extentx/2 - startboxsize) * np.sin(theta)
    y = (extenty/2 - startboxsize) * np.cos(theta)

    start_regions = [(extentx/2 + ix - startboxsize/2, # xmin
                      extentx/2 + ix + startboxsize/2, # xmax
                      extenty/2 + iy - startboxsize/2, # ymin
                      extenty/2 + iy + startboxsize/2) # ymax
                     for ix,iy in zip(x,y)]
    
    
    # Execute the 8 starting points and accumulate results
    sim = []
    for start_region, dirn in zip(start_regions, dirns):
        print(f'\n\nRunning starting region box {start_region}, track direction {dirn}')
        temp = replace(sim_object_dict, track_direction = dirn, track_start_region = start_region)
        isim = Simulator(temp)
        isim.simulate_tracks()
        sim.append(isim)
        

    # Loop throught tracks accumulating and plotting individual tracks
    pres_maps = []
    for i in range(N):
        fig, ax = sim[i].plot_simulated_tracks_altamont(show=False, in_alpha=0.1, plot_turbs=False)
        fig2, ax2, pres_map = sim[i].plot_presence_map_altamont(show=True, radius=200., minval=0.01)
        # Save figure with tracks and presence map for current direction
        sim[i].save_fig(fig, os.path.join(sim[i].fig_dir, f'simulated_tracks_alpha01_dir{theta[i]}.png'))
        sim[i].save_fig(fig2, os.path.join(sim[i].fig_dir, f'presence_map_radius200_dir{theta[i]}.png'))
        # Accumulate
        pres_maps.append(pres_map)


    # Loop through again just to plot the combined track
    for i in range(N):
        if i==0: fig, ax = sim[i].plot_simulated_tracks_altamont(show=False, in_alpha=0.1, plot_turbs=False)
        else:    _, _    = sim[i].plot_simulated_tracks_altamont(show=False, fig=fig, axs=ax, in_alpha=0.1, plot_turbs=False)
    sim[i].save_fig(fig, os.path.join(sim[i].fig_dir, f'simulated_tracks_alpha01_all{N}dir.png'))


    # Get mean presence map
    #pres_maps = np.array(pres_maps)
    summary_pres_map = np.mean(pres_maps,axis=0)
    summary_pres_map /= np.amax(summary_pres_map)

    # Save combined presence map
    fig, ax = sim[0]._plot_presence_altamont(summary_pres_map, 0.1)
    sim[0].save_fig(fig, os.path.join(sim[i].fig_dir, f'presence_map_radius200_all{N}dir.png'))


In [ ]:
# Run with improved model
runNdirections(wy_box1_improved_dict, N=8)
runNdirections(wy_box2_improved_dict, N=8)
runNdirections(wy_box3_improved_dict, N=8)
runNdirections(wy_box4_improved_dict, N=8)

In [ ]:
# Run with original model
runNdirections(wy_box1_original_dict, N=8)
runNdirections(wy_box2_original_dict, N=8)
runNdirections(wy_box3_original_dict, N=8)
runNdirections(wy_box4_original_dict, N=8)